In [2]:
import numpy as np

def viterbi_algorithm(S, Pi, E, P, Y):
    K = len(S) # Length of state space will give K
    T = len(Y) # Length of observation sequence will give T
    
    # Initialize matrices for Viterbi probabilities and paths
    viterbi_prob = np.zeros((K, T))
    viterbi_path = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        viterbi_prob[i][0] = Pi[i] * E[i][Y[0]]
        viterbi_path[i][0] = 0
    
    # Part II: Compute Viterbi probabilities and path
    for j in range(1, T):
        for i in range(K):
            viterbi_prob[i, j] = max(E[i][Y[j]] * P[k][i] * viterbi_prob[k, j-1] for k in range(K))
            viterbi_path[i, j] = np.argmax([E[i][Y[j]] * P[k][i] * viterbi_prob[k, j-1] for k in range(K)])
    
    # Part III: Re-track the most likely path
    x = []
    x_t = np.argmax(viterbi_prob[:, T-1])
    x.append(x_t)
    for j in range(T-1, 0, -1):
        x_prev = int(viterbi_path[int(x_t), j])
        x.append(x_prev)
        x_t = x_prev
        
    # Reverse path to get the correct order
    x=x[::-1]
    print(viterbi_prob)
    print(viterbi_path)
    
    return x, np.max(viterbi_prob[:, T-1])


In [3]:
import numpy as np
def forward_algorithm(S, Pi, E, P, Y, Pii=0):
    K = len(S)
    T = len(Y)
    
    forward_prob = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        forward_prob[i][0] = Pi[i] * E[i][Y[0]]
    
    # Part II: Recursion
    for j in range(1, T):
        for i in range(K):
            forward_prob[i, j] = sum(forward_prob[k, j-1] * P[k][i] * E[i][Y[j]] for k in range(K))
    
    # Part III: Termination
    total_prob = np.sum(forward_prob[:, T-1])
    return total_prob


In [4]:
# Given data
S = ['rainy', 'cloudy', 'sunny']
Pi = [0.35, 0.25, 0.4]
P = [
    [0.4, 0.2, 0.4],
    [0.2, 0.5, 0.3],
    [0.3, 0.6, 0.1]
]
E = [
    [0.8, 0.1, 0.1],
    [0.2, 0.5, 0.3],
    [0.4, 0.2, 0.4]
]
#activities = ['read', 'read', 'shop', 'play', 'shop']
activities = ['read', 'shop', 'play', 'play', 'read', 'play', 'shop', 'shop', 'shop']
#observation_indices = [0, 0, 2, 1, 2]  # Convert activities to observation indices(read->0,play->1,shop->2)
 
# (a) Find the most likely weather sequence
most_likely_weather_sequence, _ = viterbi_algorithm(S, Pi, E, P, observation_indices)
print("Most likely weather sequence:", [S[state] for state in most_likely_weather_sequence])

[[2.800000e-01 1.120000e-02 9.600000e-04 2.400000e-04 4.800000e-04
  1.536000e-04 4.915200e-05 1.966080e-06 1.179648e-06 4.718592e-08]
 [5.000000e-02 4.800000e-02 1.200000e-02 3.000000e-03 3.000000e-04
  4.320000e-05 9.216000e-06 7.372800e-06 7.372800e-07 2.654208e-07]
 [1.600000e-01 2.240000e-02 2.880000e-03 7.200000e-04 3.600000e-04
  7.680000e-05 2.457600e-05 3.932160e-06 8.847360e-07 9.437184e-08]]
[[0. 0. 1. 1. 1. 0. 0. 0. 1. 0.]
 [0. 2. 1. 1. 1. 2. 2. 2. 1. 2.]
 [0. 0. 1. 1. 1. 0. 0. 0. 1. 0.]]
Most likely weather sequence: ['sunny', 'cloudy', 'cloudy', 'cloudy', 'rainy', 'rainy', 'sunny', 'cloudy', 'sunny', 'cloudy']


In [8]:
# Given data
S = ['normal', 'alternate', 'direct']
Pi = [0.2, 0.3, 0.5]
P = [
    [0.35, 0.45, 0.2],
    [0.28, 0.32, 0.4],
    [0.75, 0.1, 0.15]
]
E = [
    [0.1, 0.9],
    [0.6, 0.4],
    [0.7, 0.3]
]
pitch_data = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
#pitch_data = ['up', 'down']

# Convert pitch data to observation indices
observation_indices = [0 if pitch == 'up' else 1 for pitch in pitch_data]

# Implement Viterbi algorithm
most_likely_control_sequence, _ = viterbi_algorithm(S, Pi, E, P, observation_indices)
print("Most likely sequence of aircraft control laws:", [S[state] for state in most_likely_control_sequence])
print(forward_algorithm(S,Pi,E,P,observation_indices,Pii=0))

[[2.00000000e-02 2.36250000e-01 7.44187500e-02 2.34419062e-02
  8.20466719e-04 2.81302875e-04 1.32915608e-04 2.29678171e-04
  8.03873600e-06 2.17045872e-05]
 [1.80000000e-01 2.30400000e-02 4.25250000e-02 1.33953750e-02
  6.32931469e-03 1.21522842e-03 2.33323857e-04 2.98654536e-05
  6.20131063e-05 7.93767760e-06]
 [3.50000000e-01 2.16000000e-02 1.41750000e-02 5.10300000e-03
  3.75070500e-03 1.77220811e-03 3.40263958e-04 2.79988628e-05
  3.21549440e-05 7.44157275e-06]]
[[0. 2. 0. 0. 0. 2. 2. 2. 0. 2.]
 [0. 1. 0. 0. 0. 1. 1. 1. 0. 1.]
 [0. 1. 0. 1. 1. 1. 1. 1. 0. 1.]]
Most likely sequence of aircraft control laws: ['direct', 'normal', 'normal', 'normal', 'alternate', 'alternate', 'direct', 'normal', 'direct', 'normal']
0.0011761669647830317
